In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if ret:
        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Hands
        results = hands.process(rgb_frame)

        # If hand landmarks are detected
        if results.multi_hand_landmarks:
            # Loop through each detected hand
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                for landmark in hand_landmarks.landmark:
                    x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        # Display the frame with hand landmarks
        cv2.imshow("Hand Detection", frame)

    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Press 'q' to quit
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize variables
prev_distance = None
zoom_factor = 1.0  # Initial zoom factor

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        # Extract landmarks for the first hand
        hand_landmarks = results.multi_hand_landmarks[0].landmark

        # Extract thumb and index finger landmarks
        thumb = (int(hand_landmarks[4].x * frame.shape[1]), int(hand_landmarks[4].y * frame.shape[0]))
        index_finger = (int(hand_landmarks[8].x * frame.shape[1]), int(hand_landmarks[8].y * frame.shape[0]))

        # Calculate distance between thumb and index finger
        distance = calculate_distance(thumb, index_finger)

        # Update zoom factor based on distance change
        if prev_distance is not None:
            zoom_factor += (distance - prev_distance) / 150  # Adjust scaling factor as needed

        prev_distance = distance

        # Draw blue dots at thumb and index finger positions
        cv2.circle(frame, thumb, 5, (255, 0, 0), -1)
        cv2.circle(frame, index_finger, 5, (255, 0, 0), -1)

    # Display the frame
    cv2.imshow('Hand Gesture Zoom', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize variables
prev_distance = None
zoom_factor = 1.0  # Initial zoom factor

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to resize image based on zoom factor
def resize_image(image, zoom_factor):
    if zoom_factor <= 0:
        zoom_factor = 1
    new_width = int(image.shape[1] * zoom_factor)
    new_height = int(image.shape[0] * zoom_factor)
    return cv2.resize(image, (new_width, new_height))

# Open webcam
cap = cv2.VideoCapture(0)

# Load the image
image_path = "akash.jpg"
image = cv2.imread(image_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        # Extract landmarks for the first hand
        hand_landmarks = results.multi_hand_landmarks[0].landmark

        # Extract thumb and index finger landmarks
        thumb = (int(hand_landmarks[4].x * frame.shape[1]), int(hand_landmarks[4].y * frame.shape[0]))
        index_finger = (int(hand_landmarks[8].x * frame.shape[1]), int(hand_landmarks[8].y * frame.shape[0]))

        # Calculate distance between thumb and index finger
        distance = calculate_distance(thumb, index_finger)

        # Update zoom factor based on distance change
        if prev_distance is not None:
            zoom_factor += (distance - prev_distance) / 150  # Adjust scaling factor as needed
            image_resized = resize_image(image, zoom_factor)

        prev_distance = distance

        # Draw blue dots at thumb and index finger positions
        cv2.circle(frame, thumb, 5, (255, 0, 0), -1)
        cv2.circle(frame, index_finger, 5, (255, 0, 0), -1)

    # Display the webcam feed
    cv2.imshow('Hand Gesture Zoom', frame)

    # Display the image if it exists
    if 'image_resized' in locals():
        cv2.imshow('Image', image_resized)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()